# Creating In-Memory Database

In [1]:
import os
import sys
from datetime import datetime
import chardet
import pandas as pd
import numpy as np

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
from utils import setup_database


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\jacob\Anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\jacob\Anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\jacob\Anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\jacob\Anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\jacob\Anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\jacob\Anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\jacob\Anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "c:\Users\jacob\Anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

Load Email Data

In [2]:
# Open CSV
with open('./emailheaders.csv', 'rb') as f:
    result = chardet.detect(f.read())

df = pd.read_csv('./emailheaders.csv', encoding=result['encoding'])

df = df.convert_dtypes()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

df.head()

,From,To,Date,Subject
0,Varja.Lagos@gastech.com.kronos,"Varja.Lagos@gastech.com.kronos, Hennie.Osvaldo...",2014-01-06 10:28:00,Patrol schedule changes
1,Brand.Tempestad@gastech.com.kronos,"Birgitta.Frente@gastech.com.kronos, Lars.Azada...",2014-01-06 10:35:00,Wellhead flow rate data
2,Isak.Baza@gastech.com.kronos,"Isak.Baza@gastech.com.kronos, Lucas.Alcazar@ga...",2014-01-06 10:43:00,RE: GT-SeismicProcessorPro Bug Report
3,Lucas.Alcazar@gastech.com.kronos,"Isak.Baza@gastech.com.kronos, Lucas.Alcazar@ga...",2014-01-06 10:50:00,RE: GT-SeismicProcessorPro Bug Report
4,Linnea.Bergen@gastech.com.kronos,"Rachel.Pantanal@gastech.com.kronos, Lars.Azada...",2014-01-06 11:00:00,RE: Upcoming birthdays


Expand email data

In [3]:
df['To'] = df['To'].str.split(',')

# Explode the 'To' column into separate rows
df = df.explode('To')

# Clean whitespace
df['From'] = df['From'].str.strip()
df['To'] = df['To'].str.strip()

In [4]:
# Remove rows where sender and recipient are the same
df = df[df['From'] != df['To']]

# Optional: reset index
df.reset_index(drop=True, inplace=True)

df.head()

,From,To,Date,Subject
0,Varja.Lagos@gastech.com.kronos,Hennie.Osvaldo@gastech.com.kronos,2014-01-06 10:28:00,Patrol schedule changes
1,Varja.Lagos@gastech.com.kronos,Loreto.Bodrogi@gastech.com.kronos,2014-01-06 10:28:00,Patrol schedule changes
2,Varja.Lagos@gastech.com.kronos,Inga.Ferro@gastech.com.kronos,2014-01-06 10:28:00,Patrol schedule changes
3,Brand.Tempestad@gastech.com.kronos,Birgitta.Frente@gastech.com.kronos,2014-01-06 10:35:00,Wellhead flow rate data
4,Brand.Tempestad@gastech.com.kronos,Lars.Azada@gastech.com.kronos,2014-01-06 10:35:00,Wellhead flow rate data


In [5]:
nodes_df = df['From'].drop_duplicates().reset_index(drop=True)

In [6]:
nodes_df = pd.DataFrame(nodes_df)

In [7]:
nodes_df_reset = nodes_df.reset_index(drop=True)
nodes_df_reset.to_excel('nodes_reset.xlsx', index=False)

In [8]:
df.to_excel('edges.xlsx', index=False)

In [9]:
# Download sample data for the Kuzudb example

# Set up the Kuzudb database connection
connection = setup_database('tmp', delete_existing=True)

# Create schema
connection.execute('CREATE NODE TABLE Person (emailAdd STRING, PRIMARY KEY (emailAdd))')
connection.execute('CREATE REL TABLE Emailed (FROM Person TO Person, timestamp TIMESTAMP, subject STRING)')

# Insert data
connection.execute('COPY Person FROM "./nodes.csv" (HEADER=TRUE)')
connection.execute('COPY Emailed FROM "./edges.csv" (HEADER=TRUE)')

Loading graph database
Removing existing database at tmp


## Running Queries

Now that your graph database is set up, you can begin querying it. This section includes seven queries, each increasing in complexity.

In [23]:
query = "MATCH (p:Person)-[e:Emailed]-(q:Person) WHERE e.timestamp >= date('2014-01-06') AND e.timestamp <= date('2014-01-19') AND toLower(p.emailAdd) CONTAINS toLower('adan') RETURN p, e, q LIMIT 100000"

result = connection.execute(query)
df = result.get_as_df()
df.to_csv("./output1.csv", index=False)

In [17]:
# Query 1: Query all nodes with the label 'Movie'. Return those movie nodes. Limit your results to 25
# result = connection.execute("MATCH (m:Movie) RETURN m LIMIT 25")
# df = result.get_as_df()
# df.head()

result = connection.execute("MATCH (n:Person)-[r:Emailed]-(m) RETURN n, r, m")
df = result.get_as_df()
df

,n,r,m
0,"{'_id': {'offset': 19, 'table': 0}, '_label': ...","{'_src': {'offset': 0, 'table': 0}, '_dst': {'...","{'_id': {'offset': 0, 'table': 0}, '_label': '..."
1,"{'_id': {'offset': 5, 'table': 0}, '_label': '...","{'_src': {'offset': 0, 'table': 0}, '_dst': {'...","{'_id': {'offset': 0, 'table': 0}, '_label': '..."
2,"{'_id': {'offset': 33, 'table': 0}, '_label': ...","{'_src': {'offset': 0, 'table': 0}, '_dst': {'...","{'_id': {'offset': 0, 'table': 0}, '_label': '..."
3,"{'_id': {'offset': 11, 'table': 0}, '_label': ...","{'_src': {'offset': 0, 'table': 0}, '_dst': {'...","{'_id': {'offset': 0, 'table': 0}, '_label': '..."
4,"{'_id': {'offset': 47, 'table': 0}, '_label': ...","{'_src': {'offset': 0, 'table': 0}, '_dst': {'...","{'_id': {'offset': 0, 'table': 0}, '_label': '..."
...,...,...,...
16365,"{'_id': {'offset': 28, 'table': 0}, '_label': ...","{'_src': {'offset': 28, 'table': 0}, '_dst': {...","{'_id': {'offset': 54, 'table': 0}, '_label': ..."
16366,"{'_id': {'offset': 24, 'table': 0}, '_label': ...","{'_src': {'offset': 24, 'table': 0}, '_dst': {...","{'_id': {'offset': 54, 'table': 0}, '_label': ..."
16367,"{'_id': {'offset': 7, 'table': 0}, '_label': '...","{'_src': {'offset': 7, 'table': 0}, '_dst': {'...","{'_id': {'offset': 54, 'table': 0}, '_label': ..."
16368,"{'_id': {'offset': 17, 'table': 0}, '_label': ...","{'_src': {'offset': 17, 'table': 0}, '_dst': {...","{'_id': {'offset': 54, 'table': 0}, '_label': ..."


In [76]:
# Query 2: Query all nodes with the label 'Movie'. Get all connected nodes to the movie nodes. Limit your results to 50
result = connection.execute("MATCH (m:Movie)-[]-() RETURN m LIMIT 50")

df = result.get_as_df()
df.head()

,m
0,"{'_id': {'offset': 1162, 'table': 0}, '_label'..."
1,"{'_id': {'offset': 1164, 'table': 0}, '_label'..."
2,"{'_id': {'offset': 1167, 'table': 0}, '_label'..."
3,"{'_id': {'offset': 1205, 'table': 0}, '_label'..."
4,"{'_id': {'offset': 1208, 'table': 0}, '_label'..."


In [18]:
# Query 3: Count the total number of nodes in the database
# Hint: Use the `COUNT` function to count the number of nodes
result = connection.execute("MATCH () RETURN COUNT(*) AS count")


s = result
df = result.get_as_df()
df.head()

,count
0,55


In [19]:
# Query 4: Query all nodes with the label 'User'. Count the degree for these nodes. Filter the nodes where the user rated more than 3 movies. Return the users and the degree
# Hint: First find all users and their ratings, then count the degree, and finally filter the results to only include users with more than 3 ratings
result = connection.execute("MATCH (u:Person)-[]-() WITH u, count(u) AS degree WHERE degree > 3 RETURN u, degree")

df = result.get_as_df()
df.head()

,u,degree
0,"{'_id': {'offset': 50, 'table': 0}, '_label': ...",242
1,"{'_id': {'offset': 15, 'table': 0}, '_label': ...",206
2,"{'_id': {'offset': 35, 'table': 0}, '_label': ...",347
3,"{'_id': {'offset': 12, 'table': 0}, '_label': ...",155
4,"{'_id': {'offset': 27, 'table': 0}, '_label': ...",267


In [91]:
# Query 5: Query all nodes with the label 'Movie'. Each node has a 'genre' attribute. Count the number of nodes per genre
# Hint: Use the `WITH` clause to group by genres and count the number of movies
result = connection.execute("MATCH (m:Movie) WITH m.genres AS genres RETURN genres, COUNT(genres)")

df = result.get_as_df()
df.head()

,genres,COUNT(genres)
0,Documentary|Drama,15
1,Drama,1058
2,Crime|Mystery|Thriller,14
3,Drama|Musical|Romance,21
4,Children|Drama|Fantasy,5


In [109]:
# Query 6: Query all nodes with the label 'Movie' and 'User', and the edge 'Rating' between movie and user. Each edge 'Rating' has a rating. Find the top 10 rated movies by average rating score
# Hint: Use the AVG clause to calculate an average. Use the `ORDER BY` clause to sort the movies by rating in descending order
result = connection.execute("MATCH (m:Movie)-[r:Rating]-(u:User) RETURN m.title, AVG(r.rating) AS avg_rating ORDER BY avg_rating DESC LIMIT 10")

df = result.get_as_df()
df.head(10)

,m.title,avg_rating
0,Alien Contamination (1980),5.0
1,Strictly Sexual (2008),5.0
2,"Valet, The (La doublure) (2006)",5.0
3,The Fox and the Hound 2 (2006),5.0
4,Ooops! Noah is Gone... (2015),5.0
5,Bloodsucking Bastards (2015),5.0
6,Cosmic Scrat-tastrophe (2015),5.0
7,Spellbound (2011),5.0
8,All Yours (2016),5.0
9,Go for Zucker! (Alles auf Zucker!) (2004),5.0


In [18]:
# Query 7: Query all nodes with the label 'Movie' and 'User', and the edge 'Rating' between movie and user. Find pairs of movies often rated by the same users
result = connection.execute("MATCH (m1:Movie)-[r1:Rating]-(u1:User), (m2:Movie)-[r2:Rating]-(u2:User) WHERE u1 = u2 AND m1.movieID < m2.movieID RETURN m1.title, m2.title, COUNT(u1) AS user_count ORDER BY user_count DESC") 

df = result.get_as_df()
df.head()

,m1.title,m2.title,user_count
0,"Shawshank Redemption, The (1994)",Forrest Gump (1994),231
1,Pulp Fiction (1994),Forrest Gump (1994),230
2,Pulp Fiction (1994),"Shawshank Redemption, The (1994)",222
3,Pulp Fiction (1994),"Silence of the Lambs, The (1991)",207
4,"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",199
